In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from ipynb.fs.full.NBAStatsWebScraper import df_NBA_complete

In [21]:
df_NBA_complete = df_NBA_complete.drop(['Tm'], axis=1)
target_col = 'Pos'
df_NBA_complete

,Player,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,2022/23 Salary
0,Precious Achiuwa,C,23,43,11,22.3,3.6,7.6,0.477,0.5,...,1.9,4.3,6.2,1.0,0.6,0.6,1.2,2.0,9.5,2840160
1,Steven Adams,C,29,42,42,27.0,3.7,6.3,0.597,0.0,...,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6,17926829
2,Bam Adebayo,C,25,62,62,34.9,8.3,15.4,0.542,0.0,...,2.5,7.0,9.5,3.2,1.2,0.8,2.5,2.9,21.2,30351780
3,Ochai Agbaji,SG,22,44,7,16.9,2.0,4.5,0.447,1.1,...,0.6,1.1,1.8,0.7,0.2,0.2,0.5,1.5,5.6,3918360
4,Santiago Aldama,PF,22,62,18,22.0,3.3,7.0,0.475,1.3,...,1.0,3.6,4.6,1.2,0.6,0.7,0.7,1.9,9.3,2094120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,McKinley Wright,PG,24,20,1,10.3,1.2,2.5,0.469,0.2,...,0.3,1.0,1.3,1.9,0.4,0.2,0.6,0.9,2.9,508891
510,Thaddeus Young,PF,34,53,9,14.9,2.0,3.7,0.545,0.1,...,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.5,8000000
511,Trae Young,PG,24,59,59,35.1,8.4,19.5,0.432,2.1,...,0.8,2.2,3.0,10.1,1.2,0.2,4.1,1.5,26.7,37096500
512,Cody Zeller,C,30,8,0,14.5,1.9,2.9,0.652,0.0,...,1.5,1.9,3.4,0.8,0.3,0.4,0.9,2.5,5.6,743922


In [22]:
dict_imputation = dict()
for col in df_NBA_complete.columns:
    if col == target_col or col == 'Player':
        continue
    dict_imputation[col] = round(df_NBA_complete[col].mean(),3)
    df_NBA_complete[col].fillna(dict_imputation[col],inplace=True)

In [23]:
dict_imputation

{'Age': 25.753,
 'G': 40.669,
 'GS': 19.276,
 'MP': 19.507,
 'FG': 3.288,
 'FGA': 6.996,
 'FG%': 0.465,
 '3P': 0.969,
 '3PA': 2.732,
 '3P%': 0.321,
 '2P': 2.318,
 '2PA': 4.266,
 '2P%': 0.536,
 'eFG%': 0.533,
 'FT': 1.439,
 'FTA': 1.848,
 'FT%': 0.749,
 'ORB': 0.858,
 'DRB': 2.633,
 'TRB': 3.489,
 'AST': 1.997,
 'STL': 0.603,
 'BLK': 0.375,
 'TOV': 1.096,
 'PF': 1.679,
 'PTS': 8.978,
 '2022/23 Salary': 8625276.165}

In [24]:
dict_normalize = dict()
for col in df_NBA_complete.columns:
    if col == target_col or col == 'Player':
        continue
    dict_normalize[col] = (df_NBA_complete[col].min(),df_NBA_complete[col].max())
    df_NBA_complete[col] = df_NBA_complete[col].apply(lambda x : (x - dict_normalize[col][0])/(dict_normalize[col][1] - dict_normalize[col][0]))
dict_normalize

{'Age': (19, 42),
 'G': (1, 68),
 'GS': (0, 68),
 'MP': (1.0, 37.5),
 'FG': (0.0, 11.3),
 'FGA': (0.0, 22.6),
 'FG%': (0.0, 1.0),
 '3P': (0.0, 5.0),
 '3PA': (0.0, 11.5),
 '3P%': (0.0, 1.0),
 '2P': (0.0, 10.2),
 '2PA': (0.0, 17.6),
 '2P%': (0.0, 1.0),
 'eFG%': (0.0, 1.0),
 'FT': (0.0, 10.1),
 'FTA': (0.0, 12.7),
 'FT%': (0.0, 1.0),
 'ORB': (0.0, 5.1),
 'DRB': (0.0, 9.7),
 'TRB': (0.0, 12.5),
 'AST': (0.0, 10.8),
 'STL': (0.0, 2.1),
 'BLK': (0.0, 3.2),
 'TOV': (0.0, 4.1),
 'PF': (0.0, 3.9),
 'PTS': (0.0, 33.3),
 '2022/23 Salary': (58493, 48070014)}

In [25]:
df_NBA_complete.head()

,Player,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,2022/23 Salary
0,Precious Achiuwa,C,0.173913,0.626866,0.161765,0.583562,0.318584,0.336283,0.477,0.10,...,0.372549,0.443299,0.496,0.092593,0.285714,0.18750,0.292683,0.512821,0.285285,0.057937
1,Steven Adams,C,0.434783,0.611940,0.617647,0.712329,0.327434,0.278761,0.597,0.00,...,1.000000,0.670103,0.920,0.212963,0.428571,0.34375,0.463415,0.589744,0.258258,0.372168
2,Bam Adebayo,C,0.260870,0.910448,0.911765,0.928767,0.734513,0.681416,0.542,0.00,...,0.490196,0.721649,0.760,0.296296,0.571429,0.25000,0.609756,0.743590,0.636637,0.630959
3,Ochai Agbaji,SG,0.130435,0.641791,0.102941,0.435616,0.176991,0.199115,0.447,0.22,...,0.117647,0.113402,0.144,0.064815,0.095238,0.06250,0.121951,0.384615,0.168168,0.080395
4,Santiago Aldama,PF,0.130435,0.910448,0.264706,0.575342,0.292035,0.309735,0.475,0.26,...,0.196078,0.371134,0.368,0.111111,0.285714,0.21875,0.170732,0.487179,0.279279,0.042399
